In [1]:
import sys
sys.path.append('../')
import torch.optim as optim
from dataset import *
from model import *
import torch.nn.init as init
from sklearn.preprocessing import MinMaxScaler
from torch.utils.tensorboard import SummaryWriter

In [2]:
s_or_d = "s"
unrelated_feature_number = 8
feature_number = 11

writer = {
    'train': SummaryWriter('../logs/MSE_L1/8train'), #必须要不同的writer
    'test': SummaryWriter('../logs/MSE_L1/8test'),
}

batchsize = 32
lambda_l1 = 0.5  # 正则化系数
# lambda_l1 = 0

criterion = nn.MSELoss()
criterion = criterion.cuda()

model = MLP(feature_number).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# adaptive learning rate
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.98)


tmp = np.load("../data/simu_20000_0.1_90_140_train.npy")
if s_or_d == 0:
    max = np.max(tmp[:, 1004])
    min = np.min(tmp[:, 1004])
else:
    max = np.max(tmp[:, 1005])
    min = np.min(tmp[:, 1005])


train_dataset = Dataset("../data/features_rand_train.npy", s_or_d, "train", unrelated_feature_number)
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
test_dataset = Dataset("../data/features_rand_test.npy", s_or_d, "test", unrelated_feature_number)
test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=True)

for epoch in range(301):
    
    # print(f"Learning Rate (Single LR): {optimizer.param_groups[0]['lr']}")
    
    model.train()

    loss_total = 0
    step = 0
    loss_pure = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()

        # print(data.shape)

        # target = (target - min) / (max - min)

        output = model(data)

        loss = criterion(output, target)
        
        loss_pure = loss_pure + loss.item()
        
        l1_regularization = torch.tensor(0.0).cuda()
        for param in model.parameters():
            l1_regularization += torch.norm(param, p=1)
            
        loss = loss + l1_regularization * lambda_l1

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total = loss_total + loss.item()
        step = step + 1
    
    # scheduler.step()
    
    writer["train"].add_scalar('FS', loss_total/step, epoch)    
    
    tmp = '../pth/tmp_%d_%.2f.pth' % (epoch, loss_total/step)
    
    if epoch % 10 == 0:
        # torch.save(model, tmp)
        print("Train_epoch:" + str(epoch) + "    MAE:" + str(loss_total/step) + "   Pure:" + str(loss_pure/step))

    loss_test = 0
    step = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.cuda(), target.cuda()
            output = model(data)

            # inv_norm
            # output = output * (max - min) + min

            loss = criterion(output, target)

            loss_test = loss_test + loss.item()
            step = step + 1

        loss_mean = loss_test / step
        
        writer["test"].add_scalar('FS', loss_mean, epoch) 
        
        if epoch % 10 == 0:
            print("Test_epoch:" + str(epoch) + "    MAE_test:" + str(loss_mean))
            
writer["train"].close()
writer["test"].close()

FileNotFoundError: [Errno 2] No such file or directory: '../data/features_rand_train.npy'